In [2]:
# Required packages
require(httr)
require(rgeos)
require(rgdal)
require(maptools)
require(tidyverse)
require(plyr)
require(reshape2)
require(RColorBrewer)
require(readr)
require(dplyr)
library(tidyr)
require(jsonlite)
require(lubridate)
require(ggplot2)

# Import data from MDCA
url = "http://wg.qa.mdca.wfp.org/mission/andrew/implementation/vamcollab/survey/responsesfile?format=csv&_k=andrew.henze&_s=13XEFQ5qorC1qazl9iruZh73OAkqierUXvZEshjkngw"
raw <- GET(url = url, content_type("text/csv"))
raw$status_code

# Read as csv
raw.content <- content(raw, as = "parsed", type = "text/csv")

# Create new column headings
colname.status.filter <- raw.content[2, 1:6]
colname.headings <- raw.content[3, 7:77]
colname.headings[is.na(colname.headings)] <- "drop"

colname.new <- cbind(colname.status.filter, colname.headings)

colnames(raw.content) <- colname.new

# Drop columns with no name
raw.content <- raw.content[, -which(names(raw.content) %in% c("drop"))]

# Drop rows if response status != Completed
raw.content <- subset(raw.content, raw.content["response status"] == "Completed") # Add "status" == "Completed" in final

# Drop columns 1-6
raw.content <- raw.content[, -c(1:6)]

flat.clean <- raw.content

head(flat.clean)

Loading required package: httr
Warning message:
"package 'httr' was built under R version 3.4.0"Loading required package: rgeos
Warning message:
"package 'rgeos' was built under R version 3.4.0"Loading required package: rgdal
Warning message:
"package 'rgdal' was built under R version 3.4.0"Loading required package: sp
Warning message:
"package 'sp' was built under R version 3.4.0"Loading required package: maptools
Warning message:
"package 'maptools' was built under R version 3.4.0"Loading required package: sp
Warning message:
"package 'sp' was built under R version 3.4.0"Loading required package: tidyverse
Warning message:
"package 'tidyverse' was built under R version 3.4.0"Loading required package: plyr
Warning message:
"package 'plyr' was built under R version 3.4.0"Loading required package: reshape2
Warning message:
"package 'reshape2' was built under R version 3.4.0"Loading required package: RColorBrewer
Warning message:
"package 'RColorBrewer' was built under R version 3.4.0"Lo

[1] 200

No encoding supplied: defaulting to UTF-8.
Warning message:
"Missing column names filled in: 'X1' [1], 'X2' [2], 'X3' [3], 'X4' [4], 'X5' [5], 'X6' [6]"Warning message:
"Duplicated column names deduplicated: 's-4-hsyxfmh' => 's-4-hsyxfmh_1' [8], 's-4-hsyxfmh' => 's-4-hsyxfmh_2' [9], 's-4-hsyxfmh' => 's-4-hsyxfmh_3' [10], 's-4-hsyxfmh' => 's-4-hsyxfmh_4' [11], 's-4-hsyxfmh' => 's-4-hsyxfmh_5' [12], 's-4-hsyxfmh' => 's-4-hsyxfmh_6' [13], 's-4-hsyxfmh' => 's-4-hsyxfmh_7' [14], 's-4-hsyxfmh' => 's-4-hsyxfmh_8' [15], 's-4-hsyxfmh' => 's-4-hsyxfmh_9' [16], 's-4-hsyxfmh' => 's-4-hsyxfmh_10' [17], 's-4-hsyxfmh' => 's-4-hsyxfmh_11' [18], 's-4-hsyxfmh' => 's-4-hsyxfmh_12' [19], 's-4-hsyxfmh' => 's-4-hsyxfmh_13' [20], 's-4-hsyxfmh' => 's-4-hsyxfmh_14' [21], 's-4-hsyxfmh' => 's-4-hsyxfmh_15' [22], 's-4-hsyxfmh' => 's-4-hsyxfmh_16' [23], 's-1-cqmyofo' => 's-1-cqmyofo_1' [25], 's-1-cqmyofo' => 's-1-cqmyofo_2' [26], 's-1-cqmyofo' => 's-1-cqmyofo_3' [27], 's-1-cqmyofo' => 's-1-cqmyofo_4' [28], 's-1-cq

_id,a/enumerator,a/respondent_id,a/site_id,a/survey_date,b/survey_agree,c/age_resp_16,c/age_respondent,d/open_ended,deviceid,...,s3/ses_wall_other,s3/sick_other,s3/sick_type,s4/aid_received,s4/aid_source,s4/aid_source_other,simserial,start,subscriberid,today
10257232,Ricky,+675 70944435,PG9022,2017-06-26,1,1.0,30.0,"Yes, continues rain has affected our food garden and shortage in the area.",ee.kobotoolbox.org:2uci7rsCMifw0Hom,...,nan,nan,nan,0.0,nan,nan,simserial not found,2017-06-26T09:33:36.000+10:00,subscriberid not found,2017-06-26
10253282,Ricky,+675 72643994,PG3706,2017-06-26,1,1.0,36.0,"Yes, we had landslide in April this year and destroyed most of our garden food and most of our staple food. Also block river and it overflow and destroyed most crops too.",ee.kobotoolbox.org:2uci7rsCMifw0Hom,...,nan,nan,nan,0.0,nan,nan,simserial not found,2017-06-26T09:18:51.000+10:00,subscriberid not found,2017-06-26
10234428,Ricky,+675 70767963,PG1850,2017-06-24,1,1.0,20.0,"Yes, we have plenty food in our community but now with hot sun for the last two months, our crops beginning to show signs of dryness.",ee.kobotoolbox.org:2uci7rsCMifw0Hom,...,nan,nan,nan,0.0,nan,nan,simserial not found,2017-06-24T14:05:45.000+10:00,subscriberid not found,2017-06-24
10234284,Ricky,+675 70009490,PG3869,2017-06-24,1,1.0,53.0,"No, enough garden food in the area, also people are making new gardens.",ee.kobotoolbox.org:2uci7rsCMifw0Hom,...,nan,nan,nan,0.0,nan,nan,simserial not found,2017-06-24T13:48:16.000+10:00,subscriberid not found,2017-06-24
10234208,Ricky,+675 70153019,PG8469,2017-06-24,1,1.0,60.0,"Yes, some garden foods looking dry since last 2 months of hot sun. Recent rain have helped us causing water tanks full again.",ee.kobotoolbox.org:2uci7rsCMifw0Hom,...,nan,nan,nan,0.0,nan,nan,simserial not found,2017-06-24T13:27:51.000+10:00,subscriberid not found,2017-06-24
10234077,Ricky,+675 71110347,PG1528,2017-06-24,1,1.0,51.0,"Yes, now we have enough garden food in our area because most people working in their gardens.",ee.kobotoolbox.org:2uci7rsCMifw0Hom,...,nan,nan,nan,0.0,nan,nan,simserial not found,2017-06-24T13:12:57.000+10:00,subscriberid not found,2017-06-24


Cleaning up KoBo questionnaire deployment bug for solar question

In [3]:
flat.clean <- as.data.frame(flat.clean)

# Change nan string to NA
flat.clean[, "s3/ses_solar"] <- as.numeric(unlist(flat.clean[, "s3/ses_solar"]))
flat.clean[, "s3/ses_solar"][is.nan(unlist(flat.clean[, "s3/ses_solar"]))] <- NA

flat.clean[, "s3/ses_television"] <- as.numeric(unlist(flat.clean[, "s3/ses_television"]))
flat.clean[, "s3/ses_television"][is.nan(unlist(flat.clean[, "s3/ses_television"]))] <- NA

# Merge ses_television with ses_solar
flat.clean[, "s3/ses_solar"][is.na(flat.clean[, "s3/ses_solar"])] = ''
flat.clean[, "s3/ses_television"][is.na(flat.clean[, "s3/ses_television"])] = ''
flat.clean[, "s3/ses_television"] <- paste(flat.clean[, "s3/ses_solar"], flat.clean[, "s3/ses_television"], sep = "")
flat.clean[, "s3/ses_solar"] <- NULL

Rename columns and reorder to match questionnaire design

In [4]:
flat.clean <- rename(flat.clean, c("s3/food_smaller_meals" = "hfias.5", "s3/food_bed_hungry" = "hfias.8",
                                   "c/age_resp_16" = "age.resp.16", "a/enumerator" = "enum",
                                   "s2/main_staple" = "main.staple", "phonenumber" = "phonenumber",
                                   "s3/food_limited" = "hfias.3", "s4/aid_source" = "aid.source",
                                   "s3/food_fewer_meals" = "hfias.6", "a/respondent_id" = "id.resp",
                                   "s3/food_normal" = "hfias.2", "s2/mortality" = "deaths", 
                                   "s2/food_avail_gender" = "food.avail.gender",
                                   "s1/gender_hoh" = "gender.hoh", "s3/food_day_night" = "hfias.9",
                                   "s2/food_supply" = "food.supply", "end" = "time.end",
                                   "s3/ses_edu_hoh" = "ses.edu", "s1/province" = "adm1",
                                   "s3/food_no_food" = "hfias.7", "s3/food_not_preferred" = "hfias.4",
                                   "start" = "time.start", "s2/rice_price" = "price.rice",
                                   "s3/food_worry" = "hfias.1", "s3/child_sick" = "child.sick",
                                   "f/respondent_knowledge" = "resp.know",
                                   "s2/main_staple_supply" = "supply.main.staple",
                                   "today" = "survey.date.auto", "s3/income_change" = "income.main.change",
                                   "simserial" = "simserial", "d/open_ended" = "open.answer",
                                   "e/call_back" = "call.back", "s3/displaced" = "displaced",
                                   "c/age_respondent" = "age.resp",
                                   "s2/main_staple_price" = "price.main.staple",
                                   "g/survey_status" = "survey.status",
                                   "a/survey_date" = "survey.date.enter",
                                   "s1/gender_respondent" = "gender.resp",
                                   "s3/ses_edu_hoh_attain" = "ses.edu.level","s1/llg" = "adm3",
                                   "subscriberid" = "id.subscriber",
                                   "s3/ses_wall_material" = "ses.wall",
                                   "s2/water_supply" = "water.supply",
                                   "s3/main_income" = "income.main",
                                   "b/survey_agree" = "survey.agree",
                                   "s2/garden_production" = "garden.prod",
                                   "s1/district" = "adm2", "s2/hunger" = "hunger",
                                   "s4/aid_received" = "aid", "a/site_id" = "site.id",
                                   "deviceid" = "id.device", "s2/famine_foods" = "famine",
                                   "_id" = "id.random", 
                                   "s2/planting_materials_supply" = "plant.supply",
                                   "s3/ses_television" = "ses.solar", "s3/sick_type" = "child.sick.type",
                                   "s3/displaced_reason" = "displaced.reason",
                                   "s3/displaced_other" = "displaced.reason.other",
                                   "s3/ses_wall_other" = "ses.wall.other",
                                   "s3/sick_other" = "child.sick.type.other",
                                   "s4/aid_source_other" = "aid.source.other",
                                   "meta/deprecatedID" = "meta", "s3/other_income" = "income.main.other",
                                   "s1/location_txt" = "adm3.other"))

flat.clean <- flat.clean %>%
  select(enum, id.resp, site.id, survey.date.enter, survey.agree, age.resp,
         age.resp.16, gender.resp, gender.hoh, adm1, adm2, adm3, adm3.other, food.supply,
         water.supply, hunger, famine, food.avail.gender, deaths, garden.prod,
         main.staple, supply.main.staple, price.main.staple, plant.supply,
         price.rice, hfias.1, hfias.2, hfias.3, hfias.4, hfias.5, hfias.6,
         hfias.7, hfias.8, hfias.9, income.main, income.main.other, income.main.change,
         displaced, displaced.reason, displaced.reason.other, child.sick,
         child.sick.type, child.sick.type.other, ses.wall, ses.wall.other,
         ses.solar, ses.edu, ses.edu.level, aid, aid.source, aid.source.other,
         open.answer, call.back, resp.know, survey.status, time.start, time.end,
         phonenumber, survey.date.auto, simserial, id.subscriber, id.random, id.device)

ERROR: Error: All arguments to `rename()` must be named.
